In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:
#set global net_buffer_length=1000000; 
#set global max_allowed_packet=1000000000;

In [3]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
2

2

In [4]:
DELETE_TABLES=False
if DELETE_TABLES:
    try:
        cmd="""DROP TABLE `%s`"""%tree_edges_table_name
        cursor.execute(cmd)
    except:
        pass
    cmd="""DROP TABLE `%s`"""%contours_lines_table_name
    cursor.execute(cmd)
2

2

In [5]:
CREATE_TABLE=False
if CREATE_TABLE:
    cmd="""CREATE TABLE `%s`(
          `id` INT NOT NULL AUTO_INCREMENT,
          `source` INT,
          `is_closed` BOOLEAN NOT NULL,
          `elevation` FLOAT NOT NULL,
          `length` FLOAT,
          `number_points` FLOAT,
          `geometry` LINESTRING NOT NULL,
          PRIMARY KEY (id)
          )
          ENGINE=InnoDB;
        """%contours_lines_table_name
    cursor.execute(cmd)
2

2

In [6]:
#ASSUME GEODATAFRAMES ARE IN UTM COORDINATES
GENERATE=False
if GENERATE:
    t1=time.time()
    file_paths=glob.glob('COURBE_1-0__SHP_LAMB93_D02A_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02A_2021/*.shp')
    file_paths+=glob.glob('COURBE_1-0__SHP_LAMB93_D02B_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02B_2021/*.shp')

    for k,file_path in enumerate(file_paths[:]):
        print(file_path)
        df=gpd.read_file(file_path) 
        df['elevation']=df['ALTITUDE']
        add_contour_lines_to_database(df,cursor,source=k)
        t3=time.time()
        cmd="""SELECT c1.id AS id_1,ST_asText(c1.geometry) AS border_1,c2.id AS id_2,ST_asText(c2.geometry) AS border_2 FROM
        (SELECT * FROM %s WHERE source!=%i) AS c1
        JOIN (SELECT * FROM %s WHERE source=%i) AS c2
        ON c1.elevation=c2.elevation AND c1.length=c2.length
        
        ;
        
        """%(contours_lines_table_name,k,contours_lines_table_name,k)
        cursor.execute(cmd)
        L=cursor.fetchall()
        t4=time.time()
        print('getting pairs took %f'%(t4-t3))
        bad_ids=[elem['id_2'] for elem in L if loads(elem['border_1'])==loads(elem['border_2'])]

        t5=time.time()
        print('comparing pairs took %f'%(t5-t4))
        if len(bad_ids)>0:
            bad_ids_string='(%s)'%', '.join([str(bad_id) for bad_id in bad_ids])
            cmd="DELETE FROM %s WHERE id IN %s"%(contours_lines_table_name,bad_ids_string)
            cursor.execute(cmd)
            t6=time.time()
            print('deleting took %f'%(t6-t5))


        


    t2=time.time()
    print(t2-t1)
2

2

In [7]:
t1=time.time()
cmd="SELECT DISTINCT elevation FROM %s ORDER BY elevation"%contours_lines_table_name
cmd=cursor.execute(cmd)
elevations=np.array([elem['elevation'] for elem in cursor.fetchall()])
t2=time.time()
print(t2-t1)

0.10340261459350586


In [8]:
MERGE=False
if MERGE:
    t1=time.time()
    for level in elevations:
        print(level)
        level_open_contours_df=get_level_contours_df(cursor,level,contours_lines_table_name,is_closed=False)
        if level_open_contours_df is not None:
            crs=level_open_contours_df.estimate_utm_crs()
            level_open_contours_df=level_open_contours_df.to_crs(crs)
            merged_contours_lines=get_contour_lines_from_elevation_df(level_open_contours_df)
            if len(merged_contours_lines)>0:
                merged_contours_df=gpd.GeoDataFrame([{'geometry':ls} for ls in merged_contours_lines],geometry='geometry',crs=crs)
                merged_contours_df['elevation']=level
                add_contour_lines_to_database(merged_contours_df,cursor,is_closed=True)

                cursor.execute(cmd)
    t2=time.time()
    print(t2-t1)
2

2

In [9]:
DELETE_EDGE_TABLE=False
if DELETE_EDGE_TABLE:
    cmd="""DROP TABLE `%s`"""%tree_edges_table_name
    cursor.execute(cmd)
2

2

In [10]:
COMPUTE_TREE=False
tree_edges_table_name="tree_edges"

if COMPUTE_TREE:
    t1=time.time()
    edges=[]
    for k,level in enumerate(elevations):
        print(level)
        nearby_levels=[level]
        if k!=0:
            nearby_levels.append(elevations[k-1])
        if k!=len(elevations)-1:
            nearby_levels.append(elevations[k+1])
            
        polys_df=get_level_contours_df(cursor,level,contours_lines_table_name,is_closed=True)
        if polys_df is not None:
            polys_df['geometry']=polys_df['geometry'].apply(lambda ls:Polygon(ls))
            crs=polys_df.estimate_utm_crs()
            polys_df=polys_df.to_crs(crs)
            polys_df['area']=polys_df['geometry'].apply(lambda poly:poly.area)

            points_dfs=[get_level_contours_df(cursor,nearby_level,contours_lines_table_name,is_closed=True) for nearby_level in nearby_levels]
            points_dfs=[df for df in points_dfs if df is not None]
            if len(points_dfs)>0:
                points_df=pd.concat(points_dfs)
                points_df=points_df.to_crs(crs)
                points_df['area']=points_df['geometry'].apply(lambda ls:Polygon(ls).area)
                points_df['geometry']=points_df['geometry'].apply(lambda ls:ls.representative_point())

                inter=gpd.overlay(polys_df,points_df,keep_geom_type=False)
                inter=inter[inter.id_1!=inter.id_2]
                
                edges+=list(zip(inter['id_1'],inter['id_2'],inter['area_1']-inter['area_2']))

    G=nx.DiGraph()
    G.add_edges_from([(u,v,{'weight':area_diff}) for u,v,area_diff in edges])
    for node in G.nodes():
        in_edges=[(u,v,d) for u,v,d in G.in_edges(node,data=True) if u!=v]
        if len(in_edges)>1:
            in_edges=sorted(in_edges,key=lambda edge:edge[2]['weight'])
            for u,v,_ in in_edges[1:]:
                G.remove_edge(u,v)
    tree_edges=G.edges()

    cmd="""CREATE TABLE `%s`(
    `begin` INT NOT NULL,
    `end` INT NOT NULL,
    FOREIGN KEY (begin) REFERENCES %s(id),
    FOREIGN KEY (end) REFERENCES %s(id) ,
    PRIMARY KEY (begin,end)
    );
        """%(tree_edges_table_name,contours_lines_table_name,contours_lines_table_name)
    
    cursor.execute(cmd)
    
    string_list=',\n'.join(['(%i,%i)'%(u,v) for (u,v) in tree_edges])
    cmd="""INSERT INTO %s (`begin`,`end`) VALUES %s;"""%(tree_edges_table_name,string_list)
    cursor.execute(cmd)
    
    t2=time.time()
    print(t2-t1)